In [1]:
base_model_path = "/home/xuzhiyu/tempModel/Baichuan-7B"
lora_model_path = "/home/xuzhiyu/llm_tune/models/baichuan_lora_sft_ds2_reh/checkpoint-366"
merge_model_path = "/home/xuzhiyu/llm_tune/models/baichuan_full_reh"
device = "cuda:4"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/home/xuzhiyu/llm_tune/models/baichuan_full_reh", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("/home/xuzhiyu/llm_tune/models/baichuan_full_reh", device_map=device, trust_remote_code=True)
inputs = tokenizer('Choose the correct answer: What is 1+1? A.2 B.3 C.4 Answer: A\n\n What is 1+2 A.2 B.3 Answer: ', return_tensors='pt')
inputs = inputs.to(device)
pred = model.generate(**inputs, max_new_tokens=64,repetition_penalty=1.1)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Choose the correct answer: What is 1+1? A.2 B.3 C.4 Answer: A

 What is 1+2 A.2 B.3 Answer: 


In [3]:
model

BaiChuanForCausalLM(
  (model): Model(
    (embed_tokens): Embedding(64000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=64000, bias=False)
)

In [4]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

#Base model on your local filesystem
base_model = AutoModelForCausalLM.from_pretrained(base_model_path, trust_remote_code=True, device_map="cuda:5")

In [5]:
inputs = inputs.to("cuda:5")

In [6]:
pred = base_model.generate(**inputs, max_new_tokens=64,repetition_penalty=1.1)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

Choose the correct answer: What is 1+1? A.2 B.3 C.4 Answer: A

 What is 1+2 A.2 B.3 Answer: 
What are you doing now, Mr. Smith? I am reading a book about history of China. Where did he go last week? He went to Beijing and Shanghai. How many people were there in your class yesterday morning? There was twenty-five students in my class this morning. Who will be our monitor today


In [14]:
inputs = tokenizer('1+1=? A.2 B.3 Answer:A\n\n 2+2=? A.6 B.4 Answer:', return_tensors='pt')
inputs = inputs.to(device)
pred = model.generate(**inputs, max_new_tokens=128,repetition_penalty=1.1)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

1+1=? A.2 B.3 Answer:A

 2+2=? A.6 B.4 Answer:B


In [15]:
inputs = inputs.to("cuda:5")
pred = base_model.generate(**inputs, max_new_tokens=128,repetition_penalty=1.1)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

1+1=? A.2 B.3 Answer:A

 2+2=? A.6 B.4 Answer:B

 5-7=? A.0 B.8 Answer:C

 9/9=? A.1 B.0 C.1 Answer:D

 1*1=? A.1 B.0 C.1 D.2 E.以上答案都不对 F.不知道 G.我也不知道 H.你猜 I.你自己选 J.随便吧 K.自己想 L.看心情 M.不告诉你 N.问老师 O.问我 P.别管 Q.算了 R.去死 S.滚蛋 T.无所谓 U.没关系 V.


In [7]:
from peft import PeftModel
merged_model = PeftModel.from_pretrained(base_model,lora_model_path)

In [8]:
merged_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BaiChuanForCausalLM(
      (model): Model(
        (embed_tokens): Embedding(64000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x DecoderLayer(
            (self_attn): Attention(
              (W_pack): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=12288, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (o_proj): lora.Linear(
                

In [9]:
merged_model = merged_model.merge_and_unload()

In [10]:
merged_model

BaiChuanForCausalLM(
  (model): Model(
    (embed_tokens): Embedding(64000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=64000, bias=False)
)